In [1]:
pip install numpy==1.24.4


In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
!pip install emoji
import emoji
%matplotlib inline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.5 MB/s eta 0:00:00


In [3]:
#importing rdkit
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit import RDConfig
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit import DataStructs
from rdkit.Chem.Subshape import SubshapeBuilder, SubshapeAligner, SubshapeObjects

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 57.7 MB/s eta 0:00:00


In [4]:
#importing sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR


In [5]:
# Import mordred and padelpy
!pip install mordred
!pip install padelpy
import padelpy

#from mordred import Calculator, descriptors
from mordred import Calculator, descriptors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.5 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176718 sha256=3b3afbc8ad17affbfc8ecf9036062fef79feaeeb366235d29b8e53650a419add
  Stored in directory: /root/.cache/pip/wheels/8b/30/0b/84e3f6775306e74cf5957ee4d16b10bf3927dcec44cc23d5f2
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linu

In [6]:
# Load dataset
sol = pd.read_csv('small_file.csv')


In [7]:
# Normalize temperature and store in new column
sol['Temp_norm'] = (sol['Temperature_K'] - sol['Temperature_K'].mean()) / sol['Temperature_K'].std()

In [8]:
# Generate RDKit Mol objects from SMILES
sol["Mol"] = sol["SMILES_Solute"].apply(Chem.MolFromSmiles)


In [9]:
def All_Mordred_descriptors(data):
    calc = Calculator(descriptors, ignore_3D=True)
    mols = [Chem.MolFromSmiles(smi) for smi in data]
    n_jobs = max(1, os.cpu_count() - 2)
    df = calc.pandas(mols, nproc=n_jobs)
    return df


In [10]:
mordred_descriptors = All_Mordred_descriptors(sol['SMILES_Solute'])

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
100%|██████████| 7999/7999 [26:39<00:00,  5.00it/s]


In [11]:
# Add temperature as a new feature
X = pd.concat([mordred_descriptors, sol[["Temp_norm"]]], axis=1)
Y = sol.iloc[:,5]      #solubility values are in 6th column


In [12]:
X.shape

(7999, 1614)

In [ ]:
'''
from sklearn.feature_selection import VarianceThreshold


# Remove low variance features
des_num = X.select_dtypes(include='number')                              #from dataframe X, select only columns having numeric values
vt = VarianceThreshold(threshold=0.1)                                    #VarianceThreshold --> method from scikit that remove features with low variance
des_reduced = des_num.loc[:, vt.fit(des_num).get_support()]              #vt.fit(des_num) apply the variance filter to des_num dataframe(consist only numerical values)
'''


In [13]:
des_num = X.select_dtypes(include='number')               #select only the features having numerical values from dataframe X
des_num.shape

(7999, 969)

In [17]:
from sklearn.feature_selection import VarianceThreshold


In [18]:
#selects and keeps only the molecular descriptors (features) that vary enough (variance > 0.1),
vt = VarianceThreshold(threshold=0.1)

# Fit
_ = vt.fit(des_num)

# Get the boolean mask
mask = vt.get_support()

des_reduced = des_num.loc[:, mask]
H=des_reduced
#X=H.drop(['FilterItLogS', 'SLogP'], axis=1)
X=H
X.shape

(7999, 563)

In [19]:
# Remove highly correlated features
correlated_features = set()
correlation_matrix = des_reduced.corr()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            correlated_features.add(correlation_matrix.columns[i])
X = des_reduced.drop(labels=correlated_features, axis=1, errors='ignore')


In [20]:
X.shape

(7999, 170)

In [21]:
# Split the data randomly using train_test_split()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=991)


In [22]:
from sklearn.linear_model import LinearRegression


# Train linear regression model
reg = LinearRegression()
reg.fit(X_train, Y_train)


LinearRegression()

In [23]:
# Predictions using trained linnear regression model(reg)
Y_pred_train = reg.predict(X_train)  #predict for training data
Y_pred_test = reg.predict(X_test)    #predict for testing data

In [24]:
# Metrics
print('Linear Regression - Train R2: %.2f' % r2_score(Y_train, Y_pred_train))
print('Linear Regression - Test R2: %.2f' % r2_score(Y_test, Y_pred_test))


Linear Regression - Train R2: 0.17
Linear Regression - Test R2: 0.15


In [25]:
print(X.head())
#print(Y.head())


   nAcid  nBase  nAromAtom  nAtom  nBridgehead  nHetero  nN  nO  nS  nF  ...  \
0      0      0          0     69            0        3   0   3   0   0  ...   
1      0      0          0     69            0        3   0   3   0   0  ...   
2      0      0          0     69            0        3   0   3   0   0  ...   
3      0      0          0     69            0        3   0   3   0   0  ...   
4      0      0          0     69            0        3   0   3   0   0  ...   

   naHRing  n6aHRing  n5ARing  nAHRing  n6AHRing  nFRing  n9FRing  nFARing  \
0        0         0        0        0         0       0        0        0   
1        0         0        0        0         0       0        0        0   
2        0         0        0        0         0       0        0        0   
3        0         0        0        0         0       0        0        0   
4        0         0        0        0         0       0        0        0   

   WPath  Temp_norm  
0   2524   0.600158  
1   25

In [26]:
print(Y.head())

0    0.010083
1    0.020100
2    0.033356
3    0.083356
4    0.233286
Name: Solubility(mol/L), dtype: float64


In [27]:
# MAE
print('MAE Train:', mean_absolute_error(Y_train, Y_pred_train))
print('MAE Test:', mean_absolute_error(Y_test, Y_pred_test))

MAE Train: 1.485495931604526
MAE Test: 1.6013666062122767


In [28]:
# Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)
Y_pred_train = model.predict(X_train)
Y_pred_test = model.predict(X_test)

In [29]:
print('Random Forest - Train R2: %.2f' % r2_score(Y_train, Y_pred_train))
print('Random Forest - Test R2: %.2f' % r2_score(Y_test, Y_pred_test))

Random Forest - Train R2: 0.94
Random Forest - Test R2: 0.76


In [30]:
# MAE
print('MAE Train:', mean_absolute_error(Y_train, Y_pred_train))
print('MAE Test:', mean_absolute_error(Y_test, Y_pred_test))


MAE Train: 0.11308718367779899
MAE Test: 0.31787788864096356


In [31]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error


In [32]:
# Train Gradient Boosting Regressor model
gbr = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=123)
gbr.fit(X_train, Y_train)

GradientBoostingRegressor(learning_rate=0.05, max_depth=4, n_estimators=300,
                          random_state=123)

In [33]:
# Predictions using trained Gradient Boosting Regressor (gbr)
Y_pred_train = gbr.predict(X_train)
Y_pred_test = gbr.predict(X_test)

In [34]:
# Metrics
print('Gradient Boosting - Train R2: %.2f' % r2_score(Y_train, Y_pred_train))
print('Gradient Boosting - Test R2: %.2f' % r2_score(Y_test, Y_pred_test))


Gradient Boosting - Train R2: 0.95
Gradient Boosting - Test R2: 0.74


In [35]:
# MAE
print('MAE Train:', mean_absolute_error(Y_train, Y_pred_train))
print('MAE Test:', mean_absolute_error(Y_test, Y_pred_test))

MAE Train: 0.41070953998732423
MAE Test: 0.5827294075488499
